# Part 1 Preprocess
* * *

In [48]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox

from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import  StandardScaler

from statsmodels.genmod.generalized_linear_model import GLM
import statsmodels.api as sm
from scipy import stats


### Loading the datasets

Create a list of all files and add the corresponding city names and day type

In [49]:
def load_data():
    data = [("/Users/lauraforerocamacho/Downloads/data/amsterdam_weekdays.csv", "Amsterdam", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/athens_weekdays.csv", "Athens","Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/barcelona_weekdays.csv", "Barcelona", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/berlin_weekdays.csv", "Berlin", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/budapest_weekdays.csv", "Budapest", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/lisbon_weekdays.csv", "Lisbon", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/London_weekdays.csv", "London", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/rome_weekdays.csv", "Rome", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/paris_weekdays.csv", "Paris", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/Vienna_weekdays.csv", "Vienna", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/amsterdam_weekends.csv", "Amsterdam", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/athens_weekends.csv", "Athens", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/barcelona_weekends.csv", "Barcelona", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/berlin_weekends.csv", "Berlin", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/budapest_weekends.csv", "Budapest", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/lisbon_weekends.csv", "Lisbon", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/London_weekends.csv", "London", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/rome_weekends.csv", "Rome", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/paris_weekends.csv", "Paris", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/Vienna_weekends.csv", "Vienna", "Weekends")
                 ]
    #merge data
    dfs = []
    for file_path, city_name, day_type in data:
        df = pd.read_csv(file_path, usecols=lambda col: col != 'Unnamed: 0')
        df = df.assign(city=city_name, type=day_type)
        dfs.append(df)
    data = pd.concat(dfs)
    return data

data= load_data()
numeric_colums= ["realSum", "person_capacity","cleanliness_rating","guest_satisfaction_overall",'bedrooms', 'dist',
       'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm', 'lng', 'lat']
categorical_colums= ["room_type","room_shared", "room_private", "host_is_superhost","multi", "biz" , 'city', 'type']

def preprocess_data(data):
    for i in categorical_colums:
        data[i] = data[i].astype('object')
        data[i].unique()
    return data
data= preprocess_data(data)

View distributin of the value

In [50]:
import seaborn as sns

def plot_distribution_features():

    fig, axes = plt.subplots(6,4,figsize=(26,20))

    # We will not plot country because it has too many categories.
    for i, c in enumerate(data.columns):
        ax = axes.reshape(-1)[i]
        if data[c].dtype.kind == 'O':
            a = sns.countplot(x=c,data=data,ax=ax)
        else:
            b = sns.histplot(x=c,data=data,ax=ax)
        t = ax.set_title(c)
    plt.tight_layout()

In [51]:
data_clean=data

def preprocess_data_LL(data_clean):
    for i in  ['Amsterdam','Athens','Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris', 'Rome','Vienna']:
        data_clean["lat_"+i]=0
        data_clean["lng_"+i]=0
    data_clean.reset_index()
    for ind in data_clean.index:
        data_clean['lat_'+data_clean['city'][ind]][ind]=data_clean['lat'][ind]
        data_clean['lng_'+data_clean['city'][ind]][ind]=data_clean['lng'][ind]
    data_clean.drop(columns=['lng','lat'],inplace=True)
    return data_clean
#data_clean = preprocess_data_LL(data_clean)
X = data_clean.loc[:,data_clean.columns != 'realSum']
y = data_clean['realSum']


In [52]:
for i in ["room_shared",	"room_private",	"host_is_superhost"]:
    X[i].replace([False, True],[0,1],inplace=True)

#X['person_capacity'] = X['person_capacity'].astype(str)
#X['bedrooms'] = X['bedrooms'].astype(str)
#,'person_capacity'
#"room_type",
for column in [ "city", "type"]:
    X_one_hot = pd.get_dummies(X[column] , dtype=int, drop_first=False)
    X = pd.concat([X, X_one_hot], axis='columns')

# drop the values
X= X.drop(["room_type", "city", "type"], axis='columns')
X_new= X


/var/folders/zv/hzjbp8kd1l103x8tgl0s00z80000gn/T/ipykernel_3968/116871885.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i].replace([False, True],[0,1],inplace=True)


In [53]:
final_train_data=X_new
final_train_data["realSum"]=y

In [54]:
final_train_data.to_csv("datapre.csv", index=False)

In [55]:
from pysal.model import spreg
from pysal.lib import weights
from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns


In [56]:
import geopandas as gpd
from pysal.model import spreg
from pysal.lib import weights
from scipy import stats
# Create a GeoDataFrame
final_train_data["geometry"] = gpd.points_from_xy(final_train_data["lng"], data["lat"])
final_train_data = gpd.GeoDataFrame(final_train_data, crs="epsg:4326")

final_train_data = final_train_data.reset_index(drop=True)

# Check the first rows
final_train_data.head()

,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,...,Budapest,Lisbon,London,Paris,Rome,Vienna,Weekdays,Weekends,realSum,geometry
0,0,1,2.0,0,1,0,10.0,93.0,1,5.022964,...,0,0,0,0,0,0,1,0,194.033698,POINT (4.90569 52.41772)
1,0,1,4.0,0,0,0,8.0,85.0,1,0.488389,...,0,0,0,0,0,0,1,0,344.245776,POINT (4.90005 52.37432)
2,0,1,2.0,0,0,1,9.0,87.0,1,5.748312,...,0,0,0,0,0,0,1,0,264.101422,POINT (4.97512 52.36103)
3,0,1,4.0,0,0,1,9.0,90.0,2,0.384862,...,0,0,0,0,0,0,1,0,433.529398,POINT (4.89417 52.37663)
4,0,1,2.0,1,0,0,10.0,98.0,1,0.544738,...,0,0,0,0,0,0,1,0,485.552926,POINT (4.90051 52.37508)


In [57]:
final_train_data["log_price"] = np.log(final_train_data["realSum"] + 0.000001)

In [58]:
final_train_data[['multi', 'biz']]=final_train_data[['multi', 'biz']].astype('float64')


In [59]:
train_data, test_data = train_test_split(final_train_data, test_size=0.2, random_state=42)

In [60]:
w = weights.KNN.from_dataframe(train_data, k=8)
w.transform = 'R'
w

/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 21 disconnected components.
  warnings.warn(message)


In [61]:
X_new.columns

Index(['room_shared', 'room_private', 'person_capacity', 'host_is_superhost',
       'multi', 'biz', 'cleanliness_rating', 'guest_satisfaction_overall',
       'bedrooms', 'dist', 'metro_dist', 'attr_index', 'attr_index_norm',
       'rest_index', 'rest_index_norm', 'lng', 'lat', 'Amsterdam', 'Athens',
       'Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris', 'Rome',
       'Vienna', 'Weekdays', 'Weekends', 'realSum', 'geometry'],
      dtype='object')

In [62]:
train_data.dtypes

room_shared                      int64
room_private                     int64
person_capacity                float64
host_is_superhost                int64
multi                          float64
biz                            float64
cleanliness_rating             float64
guest_satisfaction_overall     float64
bedrooms                         int64
dist                           float64
metro_dist                     float64
attr_index                     float64
attr_index_norm                float64
rest_index                     float64
rest_index_norm                float64
lng                            float64
lat                            float64
Amsterdam                        int64
Athens                           int64
Barcelona                        int64
Berlin                           int64
Budapest                         int64
Lisbon                           int64
London                           int64
Paris                            int64
Rome                     

In [63]:


features = ['room_shared', 'room_private', 'person_capacity', 'host_is_superhost',
       'cleanliness_rating', 'guest_satisfaction_overall','multi', 'biz',
       'bedrooms', 'dist', 'metro_dist', 'attr_index_norm',
       'rest_index_norm', 'Weekdays', 'Athens',
       'Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris', 'Rome',
       'Vienna']
target = 'realSum'


In [64]:
all_model_attributes = ["realSum"] + features
has_nans = False
for attr in all_model_attributes:
    if train_data[attr].hasnans:
        has_nans = True
print("Has missing values:", has_nans)

Has missing values: False


In [65]:
train_data = train_data.dropna(subset=all_model_attributes).copy()


In [66]:
m1 = spreg.OLS(train_data[['realSum']].values, train_data[features].values, 
                  name_y = 'realSum', name_x = features)

results = m1

print(results.summary)


REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     realSum                Number of Observations:       41365
Mean dependent var  :    279.9664                Number of Variables   :          24
S.D. dependent var  :    332.8897                Degrees of Freedom    :       41341
R-squared           :      0.2359
Adjusted R-squared  :      0.2355
Sum squared residual:3502512065.856                F-statistic           :    554.8870
Sigma-square        :   84722.480                Prob(F-statistic)     :           0
S.E. of regression  :     291.071                Log likelihood        : -293369.535
Sigma-square ML     :   84673.324                Akaike info criterion :  586787.069
S.E of regression ML:    290.9868                Schwarz criterion     :  586994.194

---------------------------------------------------------------------------

In [67]:
from sklearn.metrics import mean_squared_error

m1.intercept = m1.betas[0]  # Get the intercept from the betas array
m1.coefficients = m1.betas[1:len(m1.betas)] # Get the coefficients from the betas array
predicted_price = m1.intercept + test_data[features].dot(  m1.coefficients) 
predicted_price_2 = m1.intercept + train_data[features].dot(  m1.coefficients) 

# Calculate MSE for different models
mse_ols_1 = mean_squared_error(train_data["realSum"], predicted_price_2)
mse_ols = mean_squared_error(test_data["realSum"], predicted_price)

print(mse_ols_1)
print(mse_ols)




84673.3244495614
69139.66517001996


In [68]:
m1 = spreg.GM_Lag(train_data[['realSum']].values, train_data[features].values, w=w,
                  name_y = 'realSum', name_x = features)

results = m1

print(results.summary)


LinAlgError: Singular matrix

In [ ]:
from sklearn.metrics import mean_squared_error

m1.intercept = m1.betas[0]  # Get the intercept from the betas array
m1.coefficients = m1.betas[1:len(m1.betas)-1] # Get the coefficients from the betas array
predicted_price = m1.intercept + test_data[features].dot( m1.coefficients) 
predicted_price_2 = m1.intercept + train_data[features].dot( m1.coefficients) 

# Calculate MSE for different models
mse_ols_1 = mean_squared_error(train_data["realSum"], predicted_price_2)
mse_ols = mean_squared_error(test_data["realSum"], predicted_price)

print(mse_ols_1)
print(mse_ols)

572528151.8393285
559150800.3682243


In [ ]:
w_pool = weights.KNN.from_dataframe(train_data, k=8)
lagged = train_data.assign(w_pool=weights.spatial_lag.lag_spatial(w_pool, train_data['rest_index'].values))


/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 21 disconnected components.
  warnings.warn(message)


In [ ]:
extended_vars = features
extended_vars.append("w_pool")
train_data['w_pool']=lagged['w_pool']
m1 = spreg.OLS(train_data[['realSum']].values, train_data[extended_vars].values,
                  name_y = 'realSum', name_x = extended_vars)

results = m1

print(results.summary)


REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     realSum                Number of Observations:       41365
Mean dependent var  :    279.9664                Number of Variables   :          27
S.D. dependent var  :    332.8897                Degrees of Freedom    :       41338
R-squared           :      0.2380
Adjusted R-squared  :      0.2375
Sum squared residual:3493019364.335                F-statistic           :    496.4806
Sigma-square        :   84498.993                Prob(F-statistic)     :           0
S.E. of regression  :     290.687                Log likelihood        : -293313.404
Sigma-square ML     :   84443.838                Akaike info criterion :  586680.807
S.E of regression ML:    290.5922                Schwarz criterion     :  586913.822

---------------------------------------------------------------------------

In [ ]:
w_pool = weights.KNN.from_dataframe(test_data, k=8)
lagged = test_data.assign(w_pool=weights.spatial_lag.lag_spatial(w_pool, test_data['rest_index'].values))
test_data['w_pool']=lagged['w_pool']


/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 11 disconnected components.
  warnings.warn(message)


In [ ]:
from sklearn.metrics import mean_squared_error

m1.intercept = m1.betas[0]  # Get the intercept from the betas array
m1.coefficients = m1.betas[1:len(m1.betas)] # Get the coefficients from the betas array
predicted_price = m1.intercept + test_data[extended_vars].dot( m1.coefficients) 
predicted_price_2 = m1.intercept + train_data[extended_vars].dot( m1.coefficients) 

# Calculate MSE for different models
mse_ols_1 = mean_squared_error(train_data["realSum"], predicted_price_2)
mse_ols = mean_squared_error(test_data["realSum"], predicted_price)

print(mse_ols_1)
print(mse_ols)

84443.83813211157
69192.83142720976
